# Regressão Linear

###### Regressão Linear para prever o consumo de combustível de automóveis

A variável consumo é a variável alvo e as demais variáveis preditoras, portanto é um problema de regressão linear multipla

In [9]:
import findspark
findspark.init()

In [10]:
import numpy as np
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row # Pré Processar os dados
from pyspark.ml.linalg import Vectors # Criação de vetores para alimentar o algoritmo
from pyspark.ml.regression import LinearRegression 
from pyspark.ml.evaluation import RegressionEvaluator # Avaliador do modelo 

# Carregando os dados

In [14]:
sc = SparkContext(appName = "Lab5")

In [15]:
# Criação sa sessão Spark
apSession = SparkSession.builder.master("local").getOrCreate()

In [16]:
apSession

In [34]:
# Carregando os dados para DF Spark - ou RDD
dados = sc.textFile("C:/formacao_dataScience_DSA_DADOS/01_bigData_RealTime_Python_Spark/cap14_Apache_Spark_MLlib/Lab5/dados/dataset1.csv")

In [35]:
type(dados)

pyspark.rdd.RDD

In [36]:
# Colocando o RDD em cache. Esse processo otimiza a performance
dados.cache()

C:/formacao_dataScience_DSA_DADOS/01_bigData_RealTime_Python_Spark/cap14_Apache_Spark_MLlib/Lab5/dados/dataset1.csv MapPartitionsRDD[11] at textFile at NativeMethodAccessorImpl.java:0

In [37]:
# Número de registros
dados.count()

399

In [38]:
dados.take(5)

['consumo,numero_cilindros,capacidade,horsepower,peso,aceleracao,ano,nome',
 '30,4,79,70,2074,19.5,71,peugeot 304',
 '30,4,88,76,2065,14.5,71,fiat 124b',
 '31,4,71,65,1773,19,71,toyota corolla 1200',
 '35,4,72,69,1613,18,71,datsun 1200']

In [65]:
# Removendo o cabelaçõ
dados = dados.filter(lambda x: "numero_cilindros" not in x) # Verifica se tem a palavra 'hosrepower na lina, se tive, vai pular a linha'

In [66]:
dados.take(5)

['30,4,79,70,2074,19.5,71,peugeot 304',
 '30,4,88,76,2065,14.5,71,fiat 124b',
 '31,4,71,65,1773,19,71,toyota corolla 1200',
 '35,4,72,69,1613,18,71,datsun 1200',
 '27,4,97,60,1834,19,71,volkswagen model 111']

In [67]:
type(dados)

pyspark.rdd.PipelinedRDD

# Limpeza dos Dados

In [68]:
# Converter para DF do Spark
df_spark = dados.map(lambda x: str(x)).map(lambda w: w.split(',')).toDF()

In [69]:
# Converte DataFrame Spark para DF Pandas
df_pandas = df_spark.toPandas()

In [70]:
df_pandas.head()

,_1,_2,_3,_4,_5,_6,_7,_8
0,30,4,79,70,2074,19.5,71,peugeot 304
1,30,4,88,76,2065,14.5,71,fiat 124b
2,31,4,71,65,1773,19,71,toyota corolla 1200
3,35,4,72,69,1613,18,71,datsun 1200
4,27,4,97,60,1834,19,71,volkswagen model 111


In [71]:
# Contagem de valores nulos
df_pandas.isna().sum()

_1    0
_2    0
_3    0
_4    0
_5    0
_6    0
_7    0
_8    0
dtype: int64

In [72]:
df_pandas.isnull().values.any()

False

In [73]:
# Verifica se há alguma linha com caracter especial "?"
total = np.sum(df_pandas.apply(lambda x: x.str.contains('\?')).values)
total

6

In [74]:
df_pandas.iloc[300]

_1                                 23.9
_2                                    8
_3                                  260
_4                                   90
_5                                 3420
_6                                 22.2
_7                                   79
_8    oldsmobile cutlass salon brougham
Name: 300, dtype: object

In [75]:
# Verifica quais linhas e colunas tem o caracter especial
np.where(df_pandas.apply(lambda x: x.str.contains('\?')).values)

(array([ 48, 126, 330, 336, 354, 374], dtype=int64),
 array([3, 3, 3, 3, 3, 3], dtype=int64))

In [80]:
df_pandas.iloc[330]

_1                    40.9
_2                       4
_3                      85
_4                       ?
_5                    1835
_6                    17.3
_7                      80
_8    renault lecar deluxe
Name: 330, dtype: object

In [77]:
# Usando valor padrão para average HP (que será usado para preencher os val ausentes)
mediaHP = sc.broadcast(75.0)
# Variável global do Spark

In [78]:
# Função para limpeza dos dados
def limpaDados(inputStr):
    
    # Variável global
    global mediaHP
    
    #Lista de atributos
    attList = inputStr.split(",")
    
    # Substitui o caracter ? por um valor na coluna de indice 3
    hpValue = attList[3]
    if hpValue == "?":
        hpValue = mediaHP.value
    
    # Cria uma linha usando a função Row, limpando e convertendo os dados de string para float
    linhas = Row(consumo = float(attList[0]),
                numero_cilindros = float(attList[1]),
                capacidade = float(attList[2]),
                horsepower = float(hpValue),
                peso = float(attList[4]),
                aceleracao = float(attList[5]),
                ano = float(attList[6]),
                nome = attList[7])
    
    
    return linhas

In [79]:
dados3 = dados.map(limpaDados)
dados.cache()
dados3.take(5)

[Row(consumo=30.0, numero_cilindros=4.0, capacidade=79.0, horsepower=70.0, peso=2074.0, aceleracao=19.5, ano=71.0, nome='peugeot 304'),
 Row(consumo=30.0, numero_cilindros=4.0, capacidade=88.0, horsepower=76.0, peso=2065.0, aceleracao=14.5, ano=71.0, nome='fiat 124b'),
 Row(consumo=31.0, numero_cilindros=4.0, capacidade=71.0, horsepower=65.0, peso=1773.0, aceleracao=19.0, ano=71.0, nome='toyota corolla 1200'),
 Row(consumo=35.0, numero_cilindros=4.0, capacidade=72.0, horsepower=69.0, peso=1613.0, aceleracao=18.0, ano=71.0, nome='datsun 1200'),
 Row(consumo=27.0, numero_cilindros=4.0, capacidade=97.0, horsepower=60.0, peso=1834.0, aceleracao=19.0, ano=71.0, nome='volkswagen model 111')]

# Análise Exploratória de Dados

In [81]:
# Cria DF Spark
df_carros = apSession.createDataFrame(dados3)

In [82]:
# Estatísticas descritivas de duas variáveis (como exemplo)
df_carros.select('consumo', 'numero_cilindros').describe().show()

+-------+-----------------+-----------------+
|summary|          consumo| numero_cilindros|
+-------+-----------------+-----------------+
|  count|              398|              398|
|   mean|23.51457286432161|5.454773869346734|
| stddev|7.815984312565782|1.701004244533212|
|    min|              9.0|              3.0|
|    max|             46.6|              8.0|
+-------+-----------------+-----------------+



In [89]:
df_carros.corr('consumo', 'numero_cilindros')

-0.7753962854205548

In [91]:
# encontrando correlação entre a variavel target com as variaveis preditoras

for i in df_carros.columns:
    # Verificando se a coluna é do tipo string
    if not(isinstance(df_carros.select(i).take(1)[0][0], str)):
        print(f"Correlação da Variável Target com {i}: {df_carros.stat.corr('consumo', i)}")

Correlação da Variável Target com consumo: 1.0
Correlação da Variável Target com numero_cilindros: -0.7753962854205548
Correlação da Variável Target com capacidade: -0.8042028248058978
Correlação da Variável Target com horsepower: -0.774704152349872
Correlação da Variável Target com peso: -0.8317409332443347
Correlação da Variável Target com aceleracao: 0.4202889121016496
Correlação da Variável Target com ano: 0.5792671330833099


# Pré Processamento dos Dados

In [92]:
# Convertendo para um LabeledPoint (target, Vectro[features])
# Remove as colunas não relevantes ou com baixa correlação

def transformaVar(row):
    obj = (row["consumo"], Vectors.dense([row['peso'], row['capacidade'], row['numero_cilindros']]))
    return obj


In [93]:
# Aplica a função do RDD e cria outro RDD
dados4 = dados3.map(transformaVar)

In [94]:
dados4.take(5)

[(30.0, DenseVector([2074.0, 79.0, 4.0])),
 (30.0, DenseVector([2065.0, 88.0, 4.0])),
 (31.0, DenseVector([1773.0, 71.0, 4.0])),
 (35.0, DenseVector([1613.0, 72.0, 4.0])),
 (27.0, DenseVector([1834.0, 97.0, 4.0]))]

In [96]:
# Converte novamente o RDD para DF do SPARK
df_carros = apSession.createDataFrame(dados4, ['label', 'features'])

In [98]:
# Visualiza label (y) e atributos (x)
df_carros.select('label', 'features').show(10)

+-----+------------------+
|label|          features|
+-----+------------------+
| 30.0| [2074.0,79.0,4.0]|
| 30.0| [2065.0,88.0,4.0]|
| 31.0| [1773.0,71.0,4.0]|
| 35.0| [1613.0,72.0,4.0]|
| 27.0| [1834.0,97.0,4.0]|
| 26.0| [1955.0,91.0,4.0]|
| 24.0|[2278.0,113.0,4.0]|
| 25.0| [2126.0,97.5,4.0]|
| 23.0| [2254.0,97.0,4.0]|
| 20.0|[2408.0,140.0,4.0]|
+-----+------------------+
only showing top 10 rows



In [99]:
# Divisão de dados em Treino e teste com split 70/30
(dados_treino, dados_teste) = df_carros.randomSplit([0.7, 0.3])

In [100]:
dados_treino.count()

276

In [101]:
dados_teste.count()

122

# Machine Learning

In [102]:
# Cria o objeto
linearReg = LinearRegression()

In [103]:
# Treina o objeto com os dados de treino
modelo = linearReg.fit(dados_treino)

In [104]:
print(modelo)

LinearRegressionModel: uid=LinearRegression_3ded9c45dabd, numFeatures=3


In [110]:
# Imprimindo os coeficientes (que o modelo aprendeu)
print(f"Coeficientes: {str(modelo.coefficients)}")
print(f"Intercepto: {str(modelo.intercept)}")

Coeficientes: [-0.005784959756712458,-0.01774551096300814,0.048285998711762854]
Intercepto: 43.70612591392949


In [106]:
# Previsões com dados de teste
predictions = modelo.transform(dados_teste)

In [108]:
# Visualiza as previsões
predictions.select("features", "prediction").show()

+------------------+------------------+
|          features|        prediction|
+------------------+------------------+
|[4376.0,307.0,8.0]|13.329558142606377|
|[4382.0,318.0,8.0]|13.099647763473016|
|[4997.0,400.0,8.0]| 8.086765614128183|
|[4456.0,350.0,8.0]|12.103704390660027|
|[4951.0,455.0,8.0]| 7.376870659971509|
|[4955.0,383.0,8.0]| 8.631407610281244|
|[3988.0,350.0,8.0]| 14.81106555680146|
|[4294.0,302.0,8.0]|13.892652397471839|
|[4735.0,440.0,8.0]| 8.892604631866519|
|[4129.0,351.0,8.0]|13.977640720141995|
|[4154.0,351.0,8.0]|13.833016726224184|
|[4354.0,454.0,8.0]|10.848237145691854|
|[4457.0,318.0,8.0]|12.665775781719582|
|[4638.0,302.0,8.0]|11.902626241162757|
|[4657.0,351.0,8.0]|10.923181968597817|
|[3693.0,350.0,8.0]|16.517628685031635|
|[3761.0,400.0,8.0]|15.236975873424782|
|[3777.0,318.0,8.0]| 16.59954841628405|
|[3892.0,304.0,8.0]| 16.18271519774423|
|[3433.0,304.0,8.0]|18.838011726075248|
+------------------+------------------+
only showing top 20 rows



In [109]:
# Coeficiente de determinação R2
avaliador = RegressionEvaluator(predictionCol = "prediction", labelCol = 'label', metricName = 'r2')

In [111]:
# Resultado
avaliador.evaluate(predictions)

0.6546849977093966